# 文本分类

文本分类流程  
1、文本的清洗:pandas，正则表达式，字符串序列处理方法  
2、分词：jieba  
3、去停用词  
4、文本向量化：词袋模型，TF-IDF  
5、建模：贝叶斯  
6、模型评估：模型得分，混淆矩阵，分类报告  
7、应用：新数据的预测

数据源：http://www.sogou.com/labs/resource/ca.php

## 读取数据

In [2]:
#读取数据
import warnings
import re
warnings.filterwarnings("ignore")
import pandas as pd
import jieba
df_news = pd.read_table('./newsdata.txt',names=['category','theme','URL','content'],encoding='utf-8')

In [3]:
df_news

,category,theme,URL,content
0,汽车,新辉腾 4.2 V8 4座加长Individual版2011款 最新报价,http://auto.data.people.com.cn/model_15782/,经销商 电话 试驾/订车U憬杭州滨江区江陵路1780号4008-112233转5864#保常...
1,汽车,918 Spyder概念车,http://auto.data.people.com.cn/prdview_165423....,呼叫热线 4008-100-300 服务邮箱 kf@peopledaily.com.cn
2,汽车,日内瓦亮相 MINI性能版/概念车-1.6T引擎,http://auto.data.people.com.cn/news/story_5249...,"MINI品牌在二月曾经公布了最新的MINI新概念车Clubvan效果图,不过现在在日内瓦车展..."
3,汽车,清仓大甩卖一汽夏利N5威志V2低至3.39万,http://auto.data.people.com.cn/news/story_6144...,"清仓大甩卖!一汽夏利N5､威志V2低至3.39万=日,启新中国一汽强势推出一汽夏利N5､威志..."
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,"在今年3月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众..."
...,...,...,...,...
4995,时尚,常吃六类食物快速补充水分,http://lady.people.com.cn/GB/18248366.html,"随着天气逐渐炎热,补水变得日益重要｡据美国《跑步世界》杂志报道,喝水并不是为身体补充水分的唯..."
4996,时尚,情感:你是我的那盘菜 吃不起我走【2】,http://lady.people.com.cn/n/2012/0712/c1014-18...,"我其实不想说这些话刺激他,他也是不得已｡可是,我又该怎样说,怎样做?我只能走,离开这个伤心地..."
4997,时尚,揭秘不老女神刘晓庆的四任丈夫(图),http://lady.people.com.cn/n/2012/0730/c1014-18...,"58岁刘晓庆最新嫩照O衷诘牧跸庆绝对看不出她已经58岁了,她绝对可以秒杀刘亦菲､范冰冰这类美..."
4998,时尚,样板潮爸 时尚圈里的父亲们,http://lady.people.com.cn/GB/18215232.html,"导语:做了爸爸就是一种幸福,无论是领养还是亲生,更何况出现在影视剧中｡时尚圈永远是需要领军人..."


In [4]:
df_news.category.unique()

array(['汽车', '财经', '科技', '健康', '体育', '教育', '文化', '军事', '娱乐', '时尚'],
      dtype=object)

In [5]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  5000 non-null   object
 1   theme     5000 non-null   object
 2   URL       5000 non-null   object
 3   content   5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


## 数据清洗

In [6]:
df_news.content.values[:5]

array(['经销商 电话 试驾/订车U憬杭州滨江区江陵路1780号4008-112233转5864#保常叮00万9阒菔邪自魄白云大道北1361号;广州市天河区黄埔大道西100号富力盈泰大厦105室4008-112233转9915#保常福00万I蕉省淄博市张店区山泉路89号4008-112233转5156#保常叮00万4罅保税区黄海西三路101号4008-112233转2603#保玻埃00万L粕绞新纺锨复兴路21号4008-112233转3043#保常叮00万V泄云南昆明市度假区滇池路1268号4008-112233转7312#保常叮00万R川市兴庆区丽景北街800号4008-112233转3269#保常叮00万9尔滨市道外区先锋路469号4008-112233转2029#保矗福00万3ど呈刑煨那桂花坪街道雀园路口/星沙中南汽车世界A区05号4008-112233转7666#保常梗00万N浜菏信塘城经济开发区盘龙汽车城#矗埃埃福112233转7524#保常叮00万9阒莘禺区市广路989号(祈福食街旁)#矗埃埃福112233转9963#保常叮00万F侄新区御桥路1377号4008-112233转6337#保常福00万0不帐『戏适邪河工业区纬一路22号138.00万I虾J斜ι角江杨南路1381号4008-112233转6722#保常叮00万t奚蕉路198号4008-112233转5933#保常叮00万1本┦谐阳区北四环望京街68号4008-112233转8615#保玻福00万1本┦胁平区立汤路亚北博晟汽车汇展中心#保埃福86万=西省南昌市青山湖区科技大道599号136.00万I苄耸信劢工业区康宁路车管所对面#保常叮00万D暇┦薪宁区天元中路111号4008-112233转5501#保常叮00万3ご菏形餍戮济技术开发区长沈路4222号136.00万J家庄市北二环东路86号河北国际汽车贸易园区#矗埃埃福112233转3178#保矗福00万8壅⑶城港路99号广达车城永兴路3号136.00万I蜓羰刑西区北二中路11号4008-112233转2498#保常叮00万3啥际星嘌虼蟮溃保福负牛ㄐ挛幕宫对面)#保矗常80万A赡省沈阳市皇姑区鸭绿江街32号甲(长客总站北行1500米)#保矗福00万I钲谑新藓区罗芳立交六星汽车园进口大众4S店4008-112233转9866#保担埃00万3

### 正则表达式提取中文

In [9]:
import re
x='haklgD666FDFhlkdg更换787了个号7857895'
re.findall("[\u4e00-\u9fa5]+",x)    #查找所有的符合要求的文本，提取中文，返回的是列表

['更换', '了个号']

In [10]:
re.findall("[0-9]+",x)   #提取数字

['666', '787', '7857895']

In [11]:
re.findall("[A-z]+",x)   #提取英文

['haklgD', 'FDFhlkdg']

In [12]:
','.join(re.findall("[A-z]+",x))   #把列表恢复成字符串

'haklgD,FDFhlkdg'

In [13]:
#向量化操作
df_news[['content']].applymap(lambda x:','.join(re.findall("[\u4e00-\u9fa5]+",x)))  #提取中文字符

,content
0,"经销商,电话,试驾,订车,憬杭州滨江区江陵路,号,转,保常叮,万,阒菔邪自魄白云大道北,号,..."
1,"呼叫热线,服务邮箱"
2,"品牌在二月曾经公布了最新的,新概念车,效果图,不过现在在日内瓦车展上,品牌带来了全新的,版和..."
3,"清仓大甩卖,一汽夏利,威志,低至,万,日,启新中国一汽强势推出一汽夏利,威志,清仓大甩卖,活..."
4,"在今年,月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众..."
...,...
4995,"随着天气逐渐炎热,补水变得日益重要,据美国,跑步世界,杂志报道,喝水并不是为身体补充水分的唯..."
4996,"我其实不想说这些话刺激他,他也是不得已,可是,我又该怎样说,怎样做,我只能走,离开这个伤心地..."
4997,"岁刘晓庆最新嫩照,衷诘牧跸庆绝对看不出她已经,岁了,她绝对可以秒杀刘亦菲,范冰冰这类美女,无..."
4998,"导语,做了爸爸就是一种幸福,无论是领养还是亲生,更何况出现在影视剧中,时尚圈永远是需要领军人..."


In [7]:
df_news[['content']]=df_news[['content']].applymap(lambda x:','.join(re.findall("[\u4e00-\u9fa5]+",x))) 
df_news.head()

,category,theme,URL,content
0,汽车,新辉腾 4.2 V8 4座加长Individual版2011款 最新报价,http://auto.data.people.com.cn/model_15782/,"经销商,电话,试驾,订车,憬杭州滨江区江陵路,号,转,保常叮,万,阒菔邪自魄白云大道北,号,..."
1,汽车,918 Spyder概念车,http://auto.data.people.com.cn/prdview_165423....,"呼叫热线,服务邮箱"
2,汽车,日内瓦亮相 MINI性能版/概念车-1.6T引擎,http://auto.data.people.com.cn/news/story_5249...,"品牌在二月曾经公布了最新的,新概念车,效果图,不过现在在日内瓦车展上,品牌带来了全新的,版和..."
3,汽车,清仓大甩卖一汽夏利N5威志V2低至3.39万,http://auto.data.people.com.cn/news/story_6144...,"清仓大甩卖,一汽夏利,威志,低至,万,日,启新中国一汽强势推出一汽夏利,威志,清仓大甩卖,活..."
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,"在今年,月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众..."


In [8]:
df_news.content.values[:5]

array(['经销商,电话,试驾,订车,憬杭州滨江区江陵路,号,转,保常叮,万,阒菔邪自魄白云大道北,号,广州市天河区黄埔大道西,号富力盈泰大厦,室,转,保常福,万,蕉省淄博市张店区山泉路,号,转,保常叮,万,罅保税区黄海西三路,号,转,保玻埃,万,粕绞新纺锨复兴路,号,转,保常叮,万,泄云南昆明市度假区滇池路,号,转,保常叮,万,川市兴庆区丽景北街,号,转,保常叮,万,尔滨市道外区先锋路,号,转,保矗福,万,呈刑煨那桂花坪街道雀园路口,星沙中南汽车世界,区,号,转,保常梗,万,浜菏信塘城经济开发区盘龙汽车城,矗埃埃福,转,保常叮,万,阒莘禺区市广路,号,祈福食街旁,矗埃埃福,转,保常叮,万,侄新区御桥路,号,转,保常福,万,不帐,戏适邪河工业区纬一路,号,万,虾,斜,角江杨南路,号,转,保常叮,万,奚蕉路,号,转,保常叮,万,本,谐阳区北四环望京街,号,转,保玻福,万,本,胁平区立汤路亚北博晟汽车汇展中心,保埃福,万,西省南昌市青山湖区科技大道,号,万,苄耸信劢工业区康宁路车管所对面,保常叮,万,暇,薪宁区天元中路,号,转,保常叮,万,菏形餍戮济技术开发区长沈路,号,万,家庄市北二环东路,号河北国际汽车贸易园区,矗埃埃福,转,保矗福,万,壅,城港路,号广达车城永兴路,号,万,蜓羰刑西区北二中路,号,转,保常叮,万,啥际星嘌虼蟮溃保福负牛,挛幕宫对面,保矗常,万,赡省沈阳市皇姑区鸭绿江街,号甲,长客总站北行,米,保矗福,万,钲谑新藓区罗芳立交六星汽车园进口大众,店,转,保担埃,万,憾环城路,号,万',
       '呼叫热线,服务邮箱',
       '品牌在二月曾经公布了最新的,新概念车,效果图,不过现在在日内瓦车展上,品牌带来了全新的,版和,概念车以及更多,性能版车型亮相,停桑危伞,茫希眨危裕遥伲停粒巍,剩茫装嬖谕夤凵媳浠非常明显,其加装了空气动力学套件外观,并且针对个性的性能版车型采用了非常吸引眼球的黑色车身配红色车顶以及红色拉花的搭配,新款车型在性能上也做了改进,采用,英寸合金轮毂,高性能刹车套件以及运动化悬挂组件,配合运动版尾排以及车侧的,徽标彰显着这是一款体现性能的车型,停桑危伞,茫希眨危裕遥伲停粒巍,剩茫装嬖诙力上搭载的是一款,千瓦的,升涡轮增压引擎,最大扭矩,牛米,同时还配有全轮驱动系统,百公里加速仅需,秒,极速被限制在,或,选装,速自动变速箱

### 选择和过滤

In [9]:
#选择和过滤  选择新闻字符长度大于20的新闻
df_news['content_length']=df_news['content'].apply(lambda x:len(x)) 
df_news

,category,theme,URL,content,content_length
0,汽车,新辉腾 4.2 V8 4座加长Individual版2011款 最新报价,http://auto.data.people.com.cn/model_15782/,"经销商,电话,试驾,订车,憬杭州滨江区江陵路,号,转,保常叮,万,阒菔邪自魄白云大道北,号,...",652
1,汽车,918 Spyder概念车,http://auto.data.people.com.cn/prdview_165423....,"呼叫热线,服务邮箱",9
2,汽车,日内瓦亮相 MINI性能版/概念车-1.6T引擎,http://auto.data.people.com.cn/news/story_5249...,"品牌在二月曾经公布了最新的,新概念车,效果图,不过现在在日内瓦车展上,品牌带来了全新的,版和...",320
3,汽车,清仓大甩卖一汽夏利N5威志V2低至3.39万,http://auto.data.people.com.cn/news/story_6144...,"清仓大甩卖,一汽夏利,威志,低至,万,日,启新中国一汽强势推出一汽夏利,威志,清仓大甩卖,活...",1020
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,"在今年,月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众...",579
...,...,...,...,...,...
4995,时尚,常吃六类食物快速补充水分,http://lady.people.com.cn/GB/18248366.html,"随着天气逐渐炎热,补水变得日益重要,据美国,跑步世界,杂志报道,喝水并不是为身体补充水分的唯...",454
4996,时尚,情感:你是我的那盘菜 吃不起我走【2】,http://lady.people.com.cn/n/2012/0712/c1014-18...,"我其实不想说这些话刺激他,他也是不得已,可是,我又该怎样说,怎样做,我只能走,离开这个伤心地...",109
4997,时尚,揭秘不老女神刘晓庆的四任丈夫(图),http://lady.people.com.cn/n/2012/0730/c1014-18...,"岁刘晓庆最新嫩照,衷诘牧跸庆绝对看不出她已经,岁了,她绝对可以秒杀刘亦菲,范冰冰这类美女,无...",209
4998,时尚,样板潮爸 时尚圈里的父亲们,http://lady.people.com.cn/GB/18215232.html,"导语,做了爸爸就是一种幸福,无论是领养还是亲生,更何况出现在影视剧中,时尚圈永远是需要领军人...",1730


In [10]:
df_news=df_news[df_news['content_length']>30]  #选择字数大于30
df_news.shape

(4772, 5)

In [11]:
df_news.head()

,category,theme,URL,content,content_length
0,汽车,新辉腾 4.2 V8 4座加长Individual版2011款 最新报价,http://auto.data.people.com.cn/model_15782/,"经销商,电话,试驾,订车,憬杭州滨江区江陵路,号,转,保常叮,万,阒菔邪自魄白云大道北,号,...",652
2,汽车,日内瓦亮相 MINI性能版/概念车-1.6T引擎,http://auto.data.people.com.cn/news/story_5249...,"品牌在二月曾经公布了最新的,新概念车,效果图,不过现在在日内瓦车展上,品牌带来了全新的,版和...",320
3,汽车,清仓大甩卖一汽夏利N5威志V2低至3.39万,http://auto.data.people.com.cn/news/story_6144...,"清仓大甩卖,一汽夏利,威志,低至,万,日,启新中国一汽强势推出一汽夏利,威志,清仓大甩卖,活...",1020
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,"在今年,月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众...",579
5,汽车,悦翔两厢 1.5 AT 舒适型 2010款 最新报价,http://auto.data.people.com.cn/model_23900/,"经销商,电话,试驾,订车,仕嗍,焖市秦城区岷山路,号,万,浜菏薪岸区科技馆路特,号,万,亚市...",830


## 中文分词和去停用词

In [12]:
stopwords=pd.read_csv('./停用词.csv',header=0)
stopwords=stopwords['停用词'].tolist()
stopwords[:3]

['!', '"', '#']

In [13]:
#实现功能：分词，去停用词，去掉含有‘矗’的词，长度要大于1，恢复成用空格分开的字符串
def f(x):
    current_segment = jieba.lcut(x) #分词
    current_segment=[segment for segment in current_segment if (segment not in stopwords)&('矗' not in segment)&(len(segment)>1)]
    segments=" ".join(current_segment) #用空格分开的字符串
    return segments

In [14]:
df_news['content_clean']=df_news['content'].apply(f)  #向量化操作

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zz/8hl03_bd6sj_wrd8__z72r8r0000gn/T/jieba.cache
Loading model cost 0.958 seconds.
Prefix dict has been built successfully.


In [15]:
df_news['content_clean'].values[:5]

array(['经销商 电话 试驾 订车 杭州 滨江区 江陵 保常 自魄 白云 大道北 广州市 天河区 黄埔 大道 富力 盈泰 大厦 保常福 淄博市 张店区 山泉 保常 保税区 黄海 三路 保玻埃 纺锨 复兴路 保常 云南 昆明市 度假区 滇池 保常 川市 兴庆区 丽景 北街 保常 尔滨市 道外区 先锋 呈刑 桂花 街道 雀园 路口 星沙 中南 汽车 世界 保常 信塘城 经济 开发区 盘龙 汽车城 保常 阒莘禺 区市 广路 祈福 食街 保常 新区 御桥 保常福 不帐 戏适 邪河 工业区 一路 角江 南路 保常 奚蕉路 保常 谐阳区 北四环 望京 保玻福 平区立 汤路 亚北博晟 汽车 汇展 中心 保埃福 西省 南昌市 青山湖区 科技 大道 耸信 工业区 康宁 车管所 对面 保常 薪宁区 天元 中路 保常 菏形 戮济 技术开发区 长沈路 家庄 北二环 东路 河北 国际 汽车贸易 园区 城港路 广达 车城 永兴路 羰刑 西区 北二 中路 保常 啥际星 保福 负牛 挛幕宫 对面 赡省 沈阳市 皇姑区 鸭绿江 长客 总站 新藓区 罗芳 立交 六星 汽车 进口 大众 保担 环城路',
       '品牌 二月 公布 最新 概念车 效果图 日内瓦 车展 品牌 带来 全新 概念车 性能 车型 亮相 停桑 危伞 茫希 危裕遥 停粒巍 剩茫装 非常明显 加装 空气 动力学 套件 外观 个性 性能 车型 采用 吸引 眼球 黑色 车身 红色 车顶 红色 拉花 搭配 新款 车型 性能 改进 采用 英寸 合金 轮毂 高性能 刹车 套件 运动 悬挂 组件 配合 运动版 尾排 车侧 徽标 彰显 一款 体现 性能 车型 停桑 危伞 茫希 危裕遥 停粒巍 剩茫装 诙力 搭载 一款 千瓦 涡轮 增压 引擎 扭矩 牛米 配有 全轮 驱动 系统 百公里 加速 极速 选装 自动 变速箱 全文 来源 网上 车市',
       '清仓 甩卖 一汽 夏利 威志 低至 启新 中国 一汽 强势 推出 一汽 夏利 威志 清仓 甩卖 活动 冲击 一汽 家用 精品 轿车 上市 史无前例 谷底 销售 一汽 低至 威志 低至 全新 触底 冰点价 新威姿 一汽 威志 独家 包销 珍藏版 车型 限量 抢购 清仓 底价 促销 优惠 详细 咨询 一款 经典 国民 家用 精品 轿车 一汽 配备 安全气囊 雾灯 电子 助力 转向 系统

In [18]:
df1=df_news[['content_clean','category']]
df1.head()#去掉停用词后

,content_clean,category
0,经销商 电话 试驾 订车 杭州 滨江区 江陵 保常 自魄 白云 大道北 广州市 天河区 黄埔...,汽车
2,品牌 二月 公布 最新 概念车 效果图 日内瓦 车展 品牌 带来 全新 概念车 性能 车型 ...,汽车
3,清仓 甩卖 一汽 夏利 威志 低至 启新 中国 一汽 强势 推出 一汽 夏利 威志 清仓 甩...,汽车
4,日内瓦 车展 见到 高尔夫 家族 成员 高尔夫 敞篷版 全新 敞篷车 众多 高尔夫 车迷 关...,汽车
5,经销商 电话 试驾 订车 秦城区 岷山 菏薪 岸区 科技馆 路特 亚市 迎宾路 丹州 村前 ...,汽车


## 标签转化

In [19]:
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 10}
df1['category'] = df1['category'].map(label_mapping) #构建一个映射方法
df1

,content_clean,category
0,经销商 电话 试驾 订车 杭州 滨江区 江陵 保常 自魄 白云 大道北 广州市 天河区 黄埔...,1
2,品牌 二月 公布 最新 概念车 效果图 日内瓦 车展 品牌 带来 全新 概念车 性能 车型 ...,1
3,清仓 甩卖 一汽 夏利 威志 低至 启新 中国 一汽 强势 推出 一汽 夏利 威志 清仓 甩...,1
4,日内瓦 车展 见到 高尔夫 家族 成员 高尔夫 敞篷版 全新 敞篷车 众多 高尔夫 车迷 关...,1
5,经销商 电话 试驾 订车 秦城区 岷山 菏薪 岸区 科技馆 路特 亚市 迎宾路 丹州 村前 ...,1
...,...,...
4995,天气 炎热 补水 变得 美国 跑步 世界 杂志 报道 喝水 身体 补充 水分 唯一 方式 瓜...,10
4996,不想 刺激 只能 离开 伤心地 想起 一句 命中 终须 命中 强求 只能 安慰 随缘 担心 ...,10
4997,刘晓庆 最新 嫩照 牧跸 看不出 秒杀 刘亦菲 范冰冰 美女 名气 话题 荣誉 刘晓庆 称得...,10
4998,导语 爸爸 一种 幸福 无论是 领养 亲生 更何况 影视剧 时尚 永远 领军人物 复制 楼盘...,10


## 训练集和测试集划分

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df1['content_clean'].values, df1['category'].values, random_state=3)

In [23]:
x_train.shape

(3579,)

In [24]:
x_test.shape

(1193,)

## 提取特征

### 词袋模型

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
train_feature = vec.fit_transform(x_train) #训练集转换为向量
train_feature

<3579x85277 sparse matrix of type '<class 'numpy.int64'>'
	with 395360 stored elements in Compressed Sparse Row format>

In [26]:
test_feature = vec.transform(x_test) #测试集转换为向量
test_feature

<1193x85277 sparse matrix of type '<class 'numpy.int64'>'
	with 115156 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.naive_bayes import MultinomialNB #贝叶斯模型,多项式
classifier = MultinomialNB() 
classifier.fit(train_feature, y_train) #训练
classifier.score(test_feature, y_test) #准确率

0.8130762782900252

### TF-IDF模型

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_feature=vectorizer.fit_transform(x_train)

test_feature = vectorizer.transform(x_test)

classifier = MultinomialNB()  #贝叶斯模型
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)  

0.815590947191953

## 模型训练，超参数优化

### 网格搜索参数优化

In [29]:
from sklearn.model_selection import GridSearchCV
import numpy as np
params={'alpha':np.linspace(0,0.2,10)}
classifier = MultinomialNB()
grid_search=GridSearchCV(classifier,param_grid=params,cv=10,verbose=2,n_jobs=-1)
grid_search.fit(train_feature, y_train)
grid_search.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   11.8s finished


{'alpha': 0.11111111111111112}

## 模型评估

### 模型得分

In [32]:
grid_search.score(test_feature, y_test)#模型得分,准确率

0.8256496227996647

### 混淆矩阵

In [34]:
from sklearn import metrics
pred_test=grid_search.predict(test_feature)
print(metrics.confusion_matrix(y_test, pred_test))#混淆矩阵

[[ 87   0   0   0   0   1   0   0   0   0]
 [  0  98   5   7   2   1   2   1   1   0]
 [  3  13 116   0   1   2   0   0   1   0]
 [  0   8   0  95   1   0   0   1   1   4]
 [  0   0   0   0 120   0   0   0   1   0]
 [  0   3   0   3   0 110   4   2   8   1]
 [  0   4   1   2   0   3  73   4  33   0]
 [  0   5   0   1   4   3   4 110   2   0]
 [  0   0   0   0   2   0  13   0 117   0]
 [  0   3   3  22   2   1   1   0  18  59]]


### 分类报告

In [35]:
labels=["汽车","财经", "科技", "健康", "体育", "教育","文化","军事","娱乐","时尚"]
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_test,target_names=labels))

              precision    recall  f1-score   support

          汽车       0.97      0.99      0.98        88
          财经       0.73      0.84      0.78       117
          科技       0.93      0.85      0.89       136
          健康       0.73      0.86      0.79       110
          体育       0.91      0.99      0.95       121
          教育       0.91      0.84      0.87       131
          文化       0.75      0.61      0.67       120
          军事       0.93      0.85      0.89       129
          娱乐       0.64      0.89      0.75       132
          时尚       0.92      0.54      0.68       109

    accuracy                           0.83      1193
   macro avg       0.84      0.83      0.83      1193
weighted avg       0.84      0.83      0.82      1193



## 新数据的预测

In [36]:
new1='今年前8个月交出32259台的成绩单，冒险家不仅是品牌内销量的绝对支柱，还是紧随BBA之后最卖座的豪华紧凑型SUV，也让林肯稳坐二线豪华阵营的领头羊，在以低姿态成功打开市场后便是要扩大成果，如何让更多消费者体验到这触手可及的美式豪华？幸好林肯已经参悟透“将性价比进行到底”这通向成功之门的七字真言，那么在不久前才上市的2022款冒险家身上，“入门即豪华”又是怎么体现的呢？随我一起来看看它的变与不变。'

In [37]:
def f(x):
    current_segment = jieba.lcut(x) #分词
    current_segment=[segment for segment in current_segment if (segment not in stopwords)&('矗' not in segment)&(len(segment)>1)]
    segments=" ".join(current_segment) #用空格分开的字符串
    return segments
new_cut=f(new1)
new_cut

'交出 32259 成绩单 冒险家 品牌 销量 支柱 紧随 BBA 卖座 豪华 紧凑型 SUV 林肯 稳坐 二线 豪华 阵营 领头羊 低姿态 成功 打开 市场 便是 成果 消费者 体验 触手可及 美式 豪华 幸好 林肯 参悟 性价比 通向 成功 之门 七字 真言 不久前 上市 2022 冒险家 身上 入门 豪华 体现'

In [39]:
test_feature = vectorizer.transform([new_cut])
test_feature

<1x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [41]:
grid_search.predict(test_feature) #预测类别

array([1], dtype=int64)

In [ ]:
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 10}